In [1]:
import nltk
from nltk.corpus import gutenberg

In [2]:
#This function gets a list as input and returns the product of all numbers in the list.
def prod (ite):
    if len(ite)==0:
        return 1
    else:
        return ite[0]*prod(ite[1:])
prod ([2,7,3]) #2 x 7 x 3 = 42

42

### Simple exercises

 - What is the difference between the following two lines? Which one will give a larger value? Will this be the case for other texts?
 
`>>> sorted(set([w.lower() for w in text1]))`

`>>> sorted([w.lower() for w in set(text1)])`

In [3]:
blake_poems = gutenberg.sents("blake-poems.txt")
blake_poems

[['[', 'Poems', 'by', 'William', 'Blake', '1789', ']'], ['SONGS', 'OF', 'INNOCENCE', 'AND', 'OF', 'EXPERIENCE', 'and', 'THE', 'BOOK', 'of', 'THEL'], ...]

 - Print the first 10 sentences of the text. Print them well formatated, not just printing the list.
 
 ~~`print (blake_poems)`~~

In [4]:
#YOUR CODE HERE

for i, sent in enumerate(blake_poems):
    if i < 10:
        print(' '.join([word for word in sent]).strip())

[ Poems by William Blake 1789 ]
SONGS OF INNOCENCE AND OF EXPERIENCE and THE BOOK of THEL
SONGS OF INNOCENCE
INTRODUCTION
Piping down the valleys wild , Piping songs of pleasant glee , On a cloud I saw a child , And he laughing said to me :
" Pipe a song about a Lamb !"
So I piped with merry cheer .
" Piper , pipe that song again ;" So I piped : he wept to hear .
" Drop thy pipe , thy happy pipe ; Sing thy songs of happy cheer :!"
So I sang the same again , While he wept with joy to hear .


### Language modeling exercises

Go through the documentation of NLTK and check how bigrams, trigrams or n-grams calculators work.

In [5]:
help(nltk.ngrams)

Help on function ngrams in module nltk.util:

ngrams(sequence, n, pad_left=False, pad_right=False, left_pad_symbol=None, right_pad_symbol=None)
    Return the ngrams generated from a sequence of items, as an iterator.
    For example:
    
        >>> from nltk.util import ngrams
        >>> list(ngrams([1,2,3,4,5], 3))
        [(1, 2, 3), (2, 3, 4), (3, 4, 5)]
    
    Wrap with list for a list version of this function.  Set pad_left
    or pad_right to true in order to get additional ngrams:
    
        >>> list(ngrams([1,2,3,4,5], 2, pad_right=True))
        [(1, 2), (2, 3), (3, 4), (4, 5), (5, None)]
        >>> list(ngrams([1,2,3,4,5], 2, pad_right=True, right_pad_symbol='</s>'))
        [(1, 2), (2, 3), (3, 4), (4, 5), (5, '</s>')]
        >>> list(ngrams([1,2,3,4,5], 2, pad_left=True, left_pad_symbol='<s>'))
        [('<s>', 1), (1, 2), (2, 3), (3, 4), (4, 5)]
        >>> list(ngrams([1,2,3,4,5], 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))

Now let's take the example from the slides, and let's calculate the frequencies.

In [18]:
corpus1 = """<s> I am Sam </s>
<s> Sam I am </s>
<s> I like Sally </s>
<s> I do not like green eggs and ham </s>
<s>"""
corpus1

'<s> I am Sam </s>\n<s> Sam I am </s>\n<s> I like Sally </s>\n<s> I do not like green eggs and ham </s>\n<s>'

In [7]:
words=corpus1.split()
words

['<s>',
 'I',
 'am',
 'Sam',
 '</s>',
 '<s>',
 'Sam',
 'I',
 'am',
 '</s>',
 '<s>',
 'I',
 'like',
 'Sally',
 '</s>',
 '<s>',
 'I',
 'do',
 'not',
 'like',
 'green',
 'eggs',
 'and',
 'ham',
 '</s>']

### Conditional Frequencies

Now we want to see which are the common words that come after another word. We will combine `nltk.bigrams` and `nltk.ConditionalFreqDist` in order to get it.

Please check the documentation, and get the conditional frequencies of our toy corpus (`corpus1`).

In [8]:
#YOUR CODE HERE
cfdist = nltk.ConditionalFreqDist(bigram for bigram in nltk.bigrams(words))
cfdist.items()

dict_items([('<s>', FreqDist({'I': 3, 'Sam': 1})), ('I', FreqDist({'am': 2, 'like': 1, 'do': 1})), ('am', FreqDist({'Sam': 1, '</s>': 1})), ('Sam', FreqDist({'</s>': 1, 'I': 1})), ('</s>', FreqDist({'<s>': 3})), ('like', FreqDist({'Sally': 1, 'green': 1})), ('Sally', FreqDist({'</s>': 1})), ('do', FreqDist({'not': 1})), ('not', FreqDist({'like': 1})), ('green', FreqDist({'eggs': 1})), ('eggs', FreqDist({'and': 1})), ('and', FreqDist({'ham': 1})), ('ham', FreqDist({'</s>': 1}))])

 - What is common to observe after the word "I"? How often do those words occur? Check that!

In [9]:
cfdist['I']

FreqDist({'am': 2, 'like': 1, 'do': 1})

We all know that we can't draw conclusions from only frequencies. There are some words (usually connectors) that appear much more often tham others.

But, we need to normalize all those figures that we have. Because of that, we will calculate conditional probabilities.

### Conditional probabilities

Instead of checking just the frequency distributions, now you will build a set of probability distributions for each word in the corpus. You have to do it in a similar way, using the `nltk.ConditionalProbDist` class. Check the documentation and play with it. You can also check chapter 2.2 at the NLTK book (Bird and others, 2009).

In [10]:
from nltk.probability import ConditionalProbDist, ELEProbDist

In [24]:
cpdist = ConditionalProbDist(cfdist, ELEProbDist, 10)
cpdist

<ConditionalProbDist with 13 conditions>

In [27]:
cpdist.items()

dict_items([('<s>', <ELEProbDist based on 4 samples>), ('I', <ELEProbDist based on 4 samples>), ('am', <ELEProbDist based on 2 samples>), ('Sam', <ELEProbDist based on 2 samples>), ('</s>', <ELEProbDist based on 3 samples>), ('like', <ELEProbDist based on 2 samples>), ('Sally', <ELEProbDist based on 1 samples>), ('do', <ELEProbDist based on 1 samples>), ('not', <ELEProbDist based on 1 samples>), ('green', <ELEProbDist based on 1 samples>), ('eggs', <ELEProbDist based on 1 samples>), ('and', <ELEProbDist based on 1 samples>), ('ham', <ELEProbDist based on 1 samples>)])

In [11]:
cpdist['I'].max()

'am'

In [12]:
cpdist['I'].prob('am')

0.2777777777777778

In [13]:
cpdist['I'].prob('do')

0.16666666666666666

### Sentence's probability

You may calculate the probability of a sentence by multiplying the different conditional probabilities in the sentence. Then, we want to calculate the probability of the following sentence:

`I like green eggs`

We want to calculate the whole sentences probability,

$P(<s>, I, like, green, eggs, </s>)$

which, by using the Markov property, we can model the sentence as

$P(<s>, I, like, green, eggs, </s>) = $

$P(I | <s>) * P(like | I) * P(green | like) * P(eggs | green) * P(</s> | eggs)$


In [15]:
#YOUR CODE HERE
cpdist['<s>'].prob('I') * cpdist['I'].prob('like') * cpdist['like'].prob('green') * cpdist['green'].prob('eggs') * cpdist['eggs'].prob('</s>')

0.00028935185185185184

In [21]:

#YOUR CODE HERE
cfreq_corpus1_bigrams= nltk.ConditionalFreqDist(nltk.bigrams(words))
cfreq_corpus1_bigrams

<ConditionalFreqDist with 13 conditions>

In [31]:
#YOUR CODE HERE
#help(nltk.ConditionalProbDist)
cprob_corpus1_bigrams = nltk.ConditionalProbDist(cfreq_corpus1_bigrams, nltk.MLEProbDist)
#YOUR CODE HERE
example = "I like green eggs"
example = "<s> " + example + " </s> <s>"
example_list = example.split()
prod([cprob_corpus1_bigrams[bigram[0]].prob(bigram[1]) for bigram in nltk.bigrams(example_list)])

0.0

### Final exercise

Now, you learned about how to create simple language models. The goal now is to see how can we use them in real life. Language models are been used in the last years for a wide variety of topics.

In this last exercise we will do something similar. You will have to get two texts from project gutenberg:

 - Alice's adventures in Wonderland: 'carroll-alice.txt'
 
 - Shakespeare's Hamlet: 'shakespeare-hamlet.txt'
 
Calculate separate bigram language models and then find out some word sequences that are more probable for one author or another.

In [16]:
#YOUR CODE HERE

### Furthermore

Are you bored? Test these models but using single letters as units instead of whole words. With letters you will be able to build a quite successful language identifier.

In [17]:
#YOUR CODE HERE